In [1]:
!pip install -U nltk

Requirement already up-to-date: nltk in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (3.4.5)


In [122]:
%%bash 
conda install -c anaconda nltk=3.2.1
- y


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda


bash: line 2: y: command not found


<div class="alert alert-block alert-info"><h2 style="font-size:30px"style="padding:1px"><center>Gutenberg Challenge - Notebook Submission </center></h2><center><h4>Challenge - Part I:</h4></center> 
<p>Using any combination of Scala, Python, and Spark, find the answers to the following questions given the books in gutenberg.zip
<p>In which books does the word “truth” appear more than twice?
<p>What is the average release date? The year the book was released on project Gutenberg.
<p>What is the most common release date?
<p>What is the ratio of English books to non English books?
<p>Which book has the most dialogue between characters?
<p>Which book is the longest?</div>  
<div class="alert alert-block alert-warning"><h5> Run the first cell to load the necessary:</h5>
       <p>Modules;
       <p>Dependencies and 
       <p>Data. 

In [11]:
import os
import sys
import re

import glob

import numpy as np
import pandas as pd
import sklearn

from nltk import *
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from __future__ import unicode_literals
from six import string_types
from nltk import *
from nltk import compat
from nltk.corpus.reader.util import *
from nltk.corpus.reader.api import *
from nltk.corpus.reader.xmldocs import *
from nltk.corpus import PlaintextCorpusReader

<html>
<head><meta name="viewport" content=width="100", initial-scale="4">
<style>
* {
  box-sizing: border-box;
}
/* Create two equal columns that floats next to each other */
.column {
  float: left;
  width: 20%;
  padding: 10px;
  height: 10px; /* Should be removed. Only for demonstration */
}
/* Clear floats after the columns */
.row{
  content: "";
  display: table;
  clear: both;
}
</style></head>
<body><div class="row">
  <div class="column" style="background-color:#f2f2f2"><br>
   <div><h1><center>ETL</center></h1>
      <hp><center>Extract - Transform - Load</center><hp>
  <div class="column" style="background-color:#f2f2f2">
<div class="alert alert-block alert-info">
<h4> </h4>
    <img src="ETL.png" width="500" height="350"></div>
  <div>
</div>
</body>
</html>

In [12]:
# Simple read.  Checking the first file.
import glob
file = '11-0.txt' 

gutLs=[]

myfile = open(file,'rt')
for line in myfile:
    gutLs.append(line)
    print(line)

﻿Project Gutenberg’s Alice’s Adventures in Wonderland, by Lewis Carroll



This eBook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever.  You may copy it, give it away or

re-use it under the terms of the Project Gutenberg License included

with this eBook or online at www.gutenberg.org





Title: Alice’s Adventures in Wonderland



Author: Lewis Carroll



Posting Date: June 25, 2008 [EBook #11]

Release Date: March, 1994

Last Updated: October 6, 2016



Language: English



Character set encoding: UTF-8



*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***





















ALICE’S ADVENTURES IN WONDERLAND



Lewis Carroll



THE MILLENNIUM FULCRUM EDITION 3.0









CHAPTER I. Down the Rabbit-Hole



Alice was beginning to get very tired of sitting by her sister on the

bank, and of having nothing to do: once or twice she had peeped into the

book her sister was reading, but it had no pictures or conversations




‘I quite agree with you,’ said the Duchess; ‘and the moral of that

is--“Be what you would seem to be”--or if you’d like it put more

simply--“Never imagine yourself not to be otherwise than what it might

appear to others that what you were or might have been was not otherwise

than what you had been would have appeared to them to be otherwise.”’



‘I think I should understand that better,’ Alice said very politely, ‘if

I had it written down: but I can’t quite follow it as you say it.’



‘That’s nothing to what I could say if I chose,’ the Duchess replied, in

a pleased tone.



‘Pray don’t trouble yourself to say it any longer than that,’ said

Alice.



‘Oh, don’t talk about trouble!’ said the Duchess. ‘I make you a present

of everything I’ve said as yet.’



‘A cheap sort of present!’ thought Alice. ‘I’m glad they don’t give

birthday presents like that!’ But she did not venture to say it out

loud.



‘Thinking again?’ the Duchess asked, with another dig of her sharp

littl

<div class="alert alert-block alert-info">
<tr>
<h3><strong>1. Analyze data.</strong></h3>
Survey the data and study it before any further any steps to learn about it and make informed decisions. This step needs to occur after the data is made available and before any further ETL step. This is a simple study of the data, getting to know the data to further design the necessary steps taking into consideration the desired outcomes.<br>
    <tr>
<h5><strong>Results from this data analysis:</strong></h5>
Source: gutenberg folder – provided with the challenge in homogeneous format, all files were .txt.<br>
The directory provided contained: 761 files.<br>
File type: text.<br>
Text encoding: utf-8<br>
Languages: Multiple languages.<br>

<h5><strong>Issues found at first:<strong></h5>
Each file contains a standardized Header and Footer that needs to be cleaned out from the file.<br>
Multiple languages.<br>
Different cases, data contains lower and upper case.
<br>
<div class="alert alert-block alert-warning">
<strong><center><h3> Define desired metadata according to provided data and desired outcomes:</h3></center></strong>
<tr>
Metadata Necessary:<br>
Title: String <br>
Author: Name (First Name + Last Name)	<br>
Posting Date: Month Day, Year {EBook #number]<br>
Release Date: Month, Year<br>
Last Updated: Month, Year<br>
Language: Multiple<br>
Character set encoding: UTF - 8<br>
 </tr>
</table>

In [30]:
import re
import glob
import os

keywords = ['Title:', 'Author:', 'Release Date:', 'Language:'] # etc - you probably get those from some source
pattern = re.compile('|'.join(keywords))

for line in gutLs:
    if pattern.search(line):
        print(line)

Title: Alice’s Adventures in Wonderland

Author: Lewis Carroll

Release Date: March, 1994

Language: English



In [37]:
import glob
import os
import pandas as pd
import re

#path = 
#fids = glob.glob(os.path.join(path, '*.txt')) # All txt files in folder

files = ['11-0.txt','12-0.txt','13-0.txt'] 

# dfList =[]
# for file in files:
#     f = open(fid, 'rt')
#     text = f.read()
#     for fname in files:
#         if file in fname:
#             df = pd.read_csv(fname, sep='\t', header=None)
#             df.columns = [fname]
#             fdf = pd.concat([df, df], axis=1)
#             dfList.append(fdf)
#             fdf.head(5) # First n rows of the DataFrame
#             #fdf.tail(5) # Last n rows of the DataFrame
#             #fdf.shape # Number of rows and columns
#             fdf.info() # Index, Datatype and Memory information
#             fdf.describe() # Summary statistics for numerical columns
#                     #finaldf = pd.concat(dfList)
#                     #print(fdf)
                    
gutLs=[]

for file in files:
    text = open(file)
    for line in text:
        gutLs.append(line)
        print(line)

﻿Project Gutenberg’s Alice’s Adventures in Wonderland, by Lewis Carroll



This eBook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever.  You may copy it, give it away or

re-use it under the terms of the Project Gutenberg License included

with this eBook or online at www.gutenberg.org





Title: Alice’s Adventures in Wonderland



Author: Lewis Carroll



Posting Date: June 25, 2008 [EBook #11]

Release Date: March, 1994

Last Updated: October 6, 2016



Language: English



Character set encoding: UTF-8



*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***





















ALICE’S ADVENTURES IN WONDERLAND



Lewis Carroll



THE MILLENNIUM FULCRUM EDITION 3.0









CHAPTER I. Down the Rabbit-Hole



Alice was beginning to get very tired of sitting by her sister on the

bank, and of having nothing to do: once or twice she had peeped into the

book her sister was reading, but it had no pictures or conversations

appear to others that what you were or might have been was not otherwise

than what you had been would have appeared to them to be otherwise.”’



‘I think I should understand that better,’ Alice said very politely, ‘if

I had it written down: but I can’t quite follow it as you say it.’



‘That’s nothing to what I could say if I chose,’ the Duchess replied, in

a pleased tone.



‘Pray don’t trouble yourself to say it any longer than that,’ said

Alice.



‘Oh, don’t talk about trouble!’ said the Duchess. ‘I make you a present

of everything I’ve said as yet.’



‘A cheap sort of present!’ thought Alice. ‘I’m glad they don’t give

birthday presents like that!’ But she did not venture to say it out

loud.



‘Thinking again?’ the Duchess asked, with another dig of her sharp

little chin.



‘I’ve a right to think,’ said Alice sharply, for she was beginning to

feel a little worried.



‘Just about as much right,’ said the Duchess, ‘as pigs have to fly; and

the m--’



But here, to Ali


And once she had really frightened her old nurse by shouting suddenly in

her ear, ‘Nurse! Do let’s pretend that I’m a hungry hyaena, and you’re a

bone.’



But this is taking us away from Alice’s speech to the kitten. ‘Let’s

pretend that you’re the Red Queen, Kitty! Do you know, I think if you

sat up and folded your arms, you’d look exactly like her. Now do try,

there’s a dear!’ And Alice got the Red Queen off the table, and set it

up before the kitten as a model for it to imitate: however, the thing

didn’t succeed, principally, Alice said, because the kitten wouldn’t

fold its arms properly. So, to punish it, she held it up to the

Looking-glass, that it might see how sulky it was--‘and if you’re not

good directly,’ she added, ‘I’ll put you through into Looking-glass

House. How would you like THAT?’



‘Now, if you’ll only attend, Kitty, and not talk so much, I’ll tell you

all my ideas about Looking-glass House. First, there’s the room you can

see through the glass--that’s

‘You’re holding it upside down!’ Alice interrupted.



‘To be sure I was!’ Humpty Dumpty said gaily, as she turned it round for

him. ‘I thought it looked a little queer. As I was saying, that SEEMS

to be done right--though I haven’t time to look it over thoroughly just

now--and that shows that there are three hundred and sixty-four days

when you might get un-birthday presents--’



‘Certainly,’ said Alice.



‘And only ONE for birthday presents, you know. There’s glory for you!’



‘I don’t know what you mean by “glory,”’ Alice said.



Humpty Dumpty smiled contemptuously. ‘Of course you don’t--till I tell

you. I meant “there’s a nice knock-down argument for you!”’



‘But “glory” doesn’t mean “a nice knock-down argument,”’ Alice objected.



‘When _I_ use a word,’ Humpty Dumpty said in rather a scornful tone, ‘it

means just what I choose it to mean--neither more nor less.’



‘The question is,’ said Alice, ‘whether you CAN make words mean so many

different things.’



‘The ques


1.F.



1.F.1.  Project Gutenberg volunteers and employees expend considerable

effort to identify, do copyright research on, transcribe and proofread

public domain works in creating the Project Gutenberg-tm

collection.  Despite these efforts, Project Gutenberg-tm electronic

works, and the medium on which they may be stored, may contain

“Defects,” such as, but not limited to, incomplete, inaccurate or

corrupt data, transcription errors, a copyright or other intellectual

property infringement, a defective or damaged disk or other medium, a

computer virus, or computer codes that damage or cannot be read by

your equipment.



1.F.2.  LIMITED WARRANTY, DISCLAIMER OF DAMAGES - Except for the “Right

of Replacement or Refund” described in paragraph 1.F.3, the Project

Gutenberg Literary Archive Foundation, the owner of the Project

Gutenberg-tm trademark, and any other party distributing a Project

Gutenberg-tm electronic work under this agreement, disclaim all

liability to you for

In [36]:
import glob
import numpy as np
import os
import re

keywords = ['Author:', 'Data Released:','Title:','Release Date:', 'First Posted:', 'Language:'] # Data to keep 
pattern = re.compile(r'|'.join(keywords))

metaData=[]
for file in files:
    for line in lines:
        if pattern.search(line): # replace all unwanted substrings
            metaData.append(line)
            print(metaData)

NameError: name 'lines' is not defined

In [38]:
import glob
import os
import pandas as pd

#path = '11-0.txt' 
#fids = glob.glob(os.path.join(path, '*.txt')) # All txt files in folder
files = ['11-0.txt','12-0.txt','13-0.txt'] 

for fid in files:
    f = open(fid, 'rt')
    text = f.read()
    for fname in files:
        if fid in fname:
            df = pd.read_csv(fname, sep='\t', header=None)
            df.columns = [fname]
            fdf = pd.concat([df, df], axis=1)
            dfList.append(fdf)
            fdf.head(5) # First n rows of the DataFrame
            fdf.tail(5) # Last n rows of the DataFrame
            fdf.shape # Number of rows and columns
            fdf.info() # Index, Datatype and Memory information
            fdf.describe() # Summary statistics for numerical columns
                    #finaldf = pd.concat(dfList)
            print(fdf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2791 entries, 0 to 2790
Data columns (total 2 columns):
11-0.txt    2791 non-null object
11-0.txt    2791 non-null object
dtypes: object(2)
memory usage: 43.7+ KB
                                               11-0.txt  \
0     Project Gutenberg’s Alice’s Adventures in Wond...   
1     This eBook is for the use of anyone anywhere a...   
2     almost no restrictions whatsoever.  You may co...   
3     re-use it under the terms of the Project Guten...   
4        with this eBook or online at www.gutenberg.org   
5               Title: Alice’s Adventures in Wonderland   
6                                 Author: Lewis Carroll   
7               Posting Date: June 25, 2008 [EBook #11]   
8                             Release Date: March, 1994   
9                         Last Updated: October 6, 2016   
10                                    Language: English   
11                        Character set encoding: UTF-8   
12    *** START OF T

In [39]:
files = ['11-0.txt','12-0.txt','13-0.txt'] 
#fids = glob.glob(os.path.join(path, '*.txt'))

name = []
for file in files:
    name.append(file)
    print(name)

['11-0.txt']
['11-0.txt', '12-0.txt']
['11-0.txt', '12-0.txt', '13-0.txt']


<div class="alert alert-block alert-warning">
Tokenization and Cleaning with NLTK
The Natural Language Toolkit, or NLTK for short, Python library written for working and modeling text.

It provides good tools for loading and cleaning text that we can use to get our data ready for working with machine learning and deep learning algorithms.


A small pipeline of text preparation including:

Load the raw text.
Split into tokens.
Convert to lowercase.
Remove punctuation from each token.
Filter out remaining tokens that are not alphabetic.
Filter out tokens that are stop words.
</div>

<div class="alert alert-block alert-info">
<h2 style="font-size:35px"style="padding:5px">
Natural Language Processing </h2>
<p> The following section makes use of NLTK.</div>

In [40]:
for file in files:
    f = open(file, 'rt')
    text = f.read()
    if file in files:
        df = pd.read_csv(file, sep='\t', header=None)
        df.columns = [file]
        fdf = pd.concat([df, df], axis=1)
        dfList.append(fdf)
        fdf.head(5) # First rows of the DataFrame
        #fdf.tail(5) # Last rows of the DataFrame
        #fdf.shape # Number of rows and columns
        print(fdf)
        fdf.info() # Index, Datatype and Memory information
        fdf.describe() # Summary statistics for numerical columns
                    #finaldf = pd.concat(dfList)
                    #print(fdf)  

                                               11-0.txt  \
0     Project Gutenberg’s Alice’s Adventures in Wond...   
1     This eBook is for the use of anyone anywhere a...   
2     almost no restrictions whatsoever.  You may co...   
3     re-use it under the terms of the Project Guten...   
4        with this eBook or online at www.gutenberg.org   
5               Title: Alice’s Adventures in Wonderland   
6                                 Author: Lewis Carroll   
7               Posting Date: June 25, 2008 [EBook #11]   
8                             Release Date: March, 1994   
9                         Last Updated: October 6, 2016   
10                                    Language: English   
11                        Character set encoding: UTF-8   
12    *** START OF THIS PROJECT GUTENBERG EBOOK ALIC...   
13                     ALICE’S ADVENTURES IN WONDERLAND   
14                                        Lewis Carroll   
15                   THE MILLENNIUM FULCRUM EDITION 3.0 

In [41]:
import re
import glob
from nltk import *
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Text Normalization.

def text_normalization(file):
    for file in files:
        file = open(file, 'rt')
        text = file.read()
        text_string = text.split()

        punctuation_re = re.compile('[%s]' % re.escape(string.punctuation))
        number_re = re.compile('(\\d+)')

        sentences = sent_tokenize(text)
        words = text.split() ## split by whitespace
        words = re.split(r'\W+', text) # split the document into words.
        tokens = word_tokenize(text)     # split into words

        stop_words = stopwords.words('english')
        words = [word.lower() for word in words]    # Normalize to lowercase.  
        words = [word for word in tokens if word.isalpha()] # Remove all tokens that are not alphabetic
                                                            #alpha_num_re = re.compile("^[a-z0-9_.]+$")
    


In [ ]:
# Filter By: White space and duplicates.
    # remove unwanted space, *.split() will automatically split on whitespace and discard duplicates, 
    # the " ".join() joins the resulting list into one string.
    return " ".join(cleaned_str.split())


  
    
    # Remove blank rows and drop duplicates

In [ ]:
Clean Header
Clean Footer
Keep pnly text body

<div class="alert alert-block alert-info"><h3> CREATING A CHALLENGE CORPUS</h3> 
<p>The following section makes use of NLTK.<br>
<p> A text corpus is a large body of text. 
Many corpora are designed to contain a careful balance of material in one or more genres.</p>
Display other information about each text, by looping over all the values of fileid corresponding to the gutenberg file identifiers listed earlier and then computing statistics for each text. 
For a compact output display, we will make sure that the numbers are all integers, using int(). 

In [8]:
# Unique File IDs
from __future__ import unicode_literals
from six import string_types
from nltk import *
from nltk import compat
from nltk.corpus.reader.util import *
from nltk.corpus.reader.api import *
from nltk.corpus.reader.xmldocs import *
from nltk.corpus import PlaintextCorpusReader

corpus_root = '/Users/carol/Downloads/gutenberg/gutenberg'
challengecorpus = PlaintextCorpusReader(corpus_root, '.*')

challengecorpus.fileids()

OSError: No such file or directory: '/Users/carol/Downloads/gutenberg/gutenberg'

In [23]:
# This code snipped displays three statistics for each text: average word length, average sentence length, and the number 
# of times each vocabulary item appears in the text on average (our lexical diversity score).

for fileid in challengecorpus.fileids():
    num_chars = len(challengecorpus.raw(fileid)) 
    num_words = len(challengecorpus.words(fileid))
    print(fileid)
    print(num_chars)
    print(num_words)


NameError: name 'challengecorpus' is not defined

<div class="alert alert-block alert-info"> <h2 style="font-size:25px"style="padding:5px"> Challenge - Part II: </h2>
<p> The following section makes use of NLTK.
Design relational schema to house the following information:
* Information about a book, such as title, author, language, and raw text delineated by paragraph.
* Information about an author such as name, and what books they’ve written.

# ER Model

The ER Model has the power of expressing database entities in a conceptual hierarchical manner. As the hierarchy goes up, it generalizes the view of entities, and as we go deep in the hierarchy, it gives us the detail of every entity included. Going up in this structure is called Generalization Reverse is called Specialization Generalization and Specialization.Since the challenge asked for a relational schema to be designed the initial ER diagram will be used to develop a database schema. This schema should contain the types of data, their properties, their relationships and their organization, creating a clear view of the developed structure. This layout and its characteristics will in turn be used to justify the database type and the database management system chosen for this specific data and the desired outputs.

<p>The next goal was to use the available data from Gutenberg .txt books to create a database that fulfills the current requirements for data and has a build in structure that facilitates analysis and answering the proposed questions.
<p>A many to many relationaship is usually implemented as associative tables (also known as join table, junction table or cross-reference table).
<p>In a relational database management system, such relationships are usually implemented by means of an associative table (also known as join table, junction table or cross-reference table), say, AB with two one-to-many relationships A -> AB and B -> AB. In this case the logical primary key for AB is formed from the two foreign keys (i.e. copies of the primary keys of A and B).

#### ER – Diagram is often useful to to visually represent a data model. A common diagramming tool is the Entity- Relationship (ER) Diagram. 
* ER diagram consists of 3 elements:
1. Entities, 
2. Attributes and 
3. Relationships 

#### ENTITIES
An ENTITY is a real world item / concept that can exist on it’s own. It may be an object with physical existence ( person, house) or it may be an object with conceptual existence (company ,job, university course) 

*Entity Type: 
An entity type is a collection of similar entities, is also a category. An entity type is a group of objects, people, concepts or other items that have the same set of properties or attribute.
*Entity type defines collection of entities that have same attribute.
*Entity type in a database is defined by its name and attribute.
*Entity instance is a single occurrence of an entity type.

#### ATTRIBUTES
Attributes are the properties that describe the entities. It’s a property of an entity type. 
Example of Attributes: Book name, price, author, publisher, etc. are attributes of Book.

* Types of Attributes: 
*   Single-valued attribute 
*   Multi-valued attribute 
*   Null Attribute 
*   Simple Attribute 
*   Composite Attribute 
*   Key Attribute 
*   Derived Attribute 
*   Attribute Domain

#### RELATIONSHIPS
A relationship is an association of entities showing how the entities are related with each other.The relationship(s) between the entities must be defined to determine the relating field(s).  

The examples of relationship are: 
• Teaching is the relationship between Lecturer and Student. 
• Treatment is the relationship between Doctor and Patient.

Types of Relationship or Multiplicity: 
Number or range of possible occurrences of an entity type in relation to another entity type, also called Cardinality, which is the number of entity type, which is involved in a relationship. (CARDINALITY) 
1.One-to-One (1 : 1) 
2.One-to-Many (1 : N) 3. Many-to-One (N : 1) 
3.Many-to-Many (M : N)

Relating Fields & Relationships 
The MAIN relationship between the author and book entity entities.
* Many to Many Relationship.
* Possibly many X’s for each Y and many Y’s for each X.  Thus the relationship between the entities BOOK and AUTHOR can be seem as Many_to_Many (M:N). 

Models can be used to help evaluate the truth or falsity English sentences, and because of it are used as a method to represent meaning. The meaning arises from the language due to its formal and structural relationships; these relationships represent a certain degree of closeness to each other and thus define connectives.

Sentences carry meaning and associations; this is why a collection of sentences that present a certain degree of association/relationship either structurally or semantically forms a paragraph.

Arguments are used to propose these relationships and need to be logically based, which means they need to be 'syntactic valid'.

<p> A BOOK is written by one or more AUTHORS. </p>

In [27]:
# M:N Relationship
from IPython.display import Image
Image(filename='Books_Author.png', width='400') 

FileNotFoundError: [Errno 2] No such file or directory: 'Books_Author.png'

In [28]:
<div class="alert alert-block alert-info"> 
A BOOK is written by one or more AUTHORS.<p> 
One AUTHOR can write one or more BOOKS.<p>

An AUTHOR can write in multiple Genres.<p>
A BOOK can be classified by more than one genre.<p>

SyntaxError: invalid syntax (<ipython-input-28-2a46bd5d711c>, line 1)

### Entities:
* Book 
* Author 
* Genre 
* Obs:
    
### Book Attributes:
* Type:
* Language:
* Title:
       
### Author Attributes:
* Name ...

### Genre Attributes:
* Name

### Relationships
* Many to Many Relationship.
* Possibly many X’s for each Y and many Y’s for each X.  Thus the relationship between the entities BOOK and AUTHOR can be seem as Many_to_Many (M:N). .</td>

An ER model builds a storage-type semantic representations of the relationships that are contained by the model and clearly represent it.

# ERD
from IPython.display import Image
Image(filename='ERD01.png', width='1800', height='1200') 

Model building tries to create a new model, given some set of sentences. If it succeeds, then we know that the set is consistent, since we have an existence proof of the model.
The general form of this valuation should be familiar : it contains some individual constants and some other predicates, each with an appropriate kind of value asociated to them.

--> From the senteces we can infer the relationship-connection and from the constants and predicates the Entities and their Attributes can also be infered.


Principle of Compositionality: The meaning of a whole is a function of the meanings of the parts and of the way they are syntactically combined. --Akin to Cardinality.

In [29]:
Data Lake

SyntaxError: invalid syntax (<ipython-input-29-ab5c30fb14f3>, line 1)